# Hacker News

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

We're specifically interested in posts whose titles begin with either Ask HN or Show HN. Users submit Ask HN posts to ask the Hacker News community a specific question. Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting.

In [1]:
from csv import reader
opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)

for row in hn[:5]:
    print(row)
    print('\n')
    


['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']




Notice that the first list in the inner lists contains the column headers, and the lists after contain the data for one row. In order to analyze our data, we need to first remove the row containing the column headers. Let's remove that first row next.

In [2]:
headers = hn[0]
hn = hn[1:]
print(headers)
print('\n')
for row in hn[:5]:
    print(row)
    print('\n')

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']




Now that we've removed the headers from hn, we're ready to filter our data. Since we're only concerned with post titles beginning with Ask HN or Show HN, we'll create new lists of lists containing just the data for those titles. 

In [3]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    title = title.lower()
    if title.startswith('ask hn'):
        ask_posts.append(row)
    elif title.startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


Ask posts : 1744
Show posts : 1162
Other posts : 17194

Let's print some rows of each list to check if it has been done correctly.


In [4]:
print(ask_posts[:2])

[['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55'], ['10610020', 'Ask HN: Am I the only one outraged by Twitter shutting down share counts?', '', '28', '29', 'tkfx', '11/22/2015 13:43']]


In [5]:
print(show_posts[:2])

[['10627194', 'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform', 'https://iot.seeed.cc', '26', '22', 'kfihihc', '11/25/2015 14:03'], ['10646440', 'Show HN: Something pointless I made', 'http://dn.ht/picklecat/', '747', '102', 'dhotson', '11/29/2015 22:46']]


In [6]:
print(other_posts[:2])

[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']]


Next, let's determine if ask posts or show posts receive more comments on average.

In [7]:
total_ask_comments = 0

for row in ask_posts:
    num_comments = row[4]
    total_ask_comments += int(num_comments)
    
avg_ask_comments = total_ask_comments / len(ask_posts)
print(round(avg_ask_comments,2))
    

14.04


In [8]:
total_show_comments = 0

for row in show_posts:
    num_comments = row[4]
    total_show_comments += int(num_comments)
    
avg_show_comments = total_show_comments / len(show_posts)
print(round(avg_show_comments,2))

10.32


On average, ask posts receive almost 4 comments more than show post.

Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts. 

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

- Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
- Calculate the average number of comments ask posts receive by hour created.


### The amount of ask posts and comments by hour created

Let's created a dictionary that contains the hour and frequency.

We'll use the datetime module to work with the data in the created_at column. We can use the datetime.strptime() constructor to parse dates stored as strings and return datetime objects:

In [9]:
import datetime as dt
result_list = []  #this will be a list of lists

for row in ask_posts:
    created_at = row[-1]
    num_comments = int(row[4])
    result_list.append([created_at, num_comments])
    
print(result_list[:5])
print(len(result_list))

[['8/16/2016 9:55', 6], ['11/22/2015 13:43', 29], ['5/2/2016 10:14', 1], ['8/2/2016 14:20', 3], ['10/15/2015 16:38', 17]]
1744


In [10]:
counts_by_hour = {} 
comments_by_hour = {}

for row in result_list:
    comments = int(row[1])
    date_string = row[0]
    date = dt.datetime.strptime(date_string, "%m/%d/%Y %H:%M")# to parse the date and create a datetime object
    hour = dt.datetime.strftime(date, "%H")# format datetime to string for hour only
    if hour in counts_by_hour:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += comments
        
    else:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = comments
        
print(counts_by_hour)
print('\n')
print(comments_by_hour)
    
    

{'19': 110, '23': 68, '16': 108, '22': 71, '20': 80, '06': 44, '04': 47, '07': 34, '03': 54, '01': 60, '13': 85, '12': 73, '10': 59, '00': 55, '02': 58, '15': 116, '11': 58, '14': 107, '08': 48, '18': 109, '09': 45, '05': 46, '17': 100, '21': 109}


{'19': 1188, '23': 543, '16': 1814, '22': 479, '20': 1722, '06': 397, '04': 337, '07': 267, '03': 421, '01': 683, '13': 1253, '12': 687, '10': 793, '00': 447, '02': 1381, '15': 4477, '11': 641, '14': 1416, '08': 492, '18': 1439, '09': 251, '05': 464, '17': 1146, '21': 1745}


We can see the highest number of comments were posted around 4pm to 6pm. One possible reason for that is because most people are off work and have time to read and post comments when they are travelling home.

### The average number of comments ask posts receive by hour created.

we'll use these two dictionaries to calculate the average number of comments for posts created during each hour of the day. 

In [13]:
avg_by_hour = []

for comment in comments_by_hour:
    average = comments_by_hour[comment]/counts_by_hour[comment]
    avg_by_hour.append([comment, round(average, 2)])
    
avg_by_hour

[['19', 10.8],
 ['23', 7.99],
 ['16', 16.8],
 ['22', 6.75],
 ['20', 21.52],
 ['06', 9.02],
 ['04', 7.17],
 ['07', 7.85],
 ['03', 7.8],
 ['01', 11.38],
 ['13', 14.74],
 ['12', 9.41],
 ['10', 13.44],
 ['00', 8.13],
 ['02', 23.81],
 ['15', 38.59],
 ['11', 11.05],
 ['14', 13.23],
 ['08', 10.25],
 ['18', 13.2],
 ['09', 5.58],
 ['05', 10.09],
 ['17', 11.46],
 ['21', 16.01]]

Let's sort the list of lists and printing the five highest values in a format that's easier to read. 

First we need to swap the columns to apply sorted function.

In [16]:
# create a list that equals avg_by_hour with swapped columns

swap_avg_by_hour = []
for row in avg_by_hour:
    new_row1 = row[0]
    new_row0 = row[1]
    swap_avg_by_hour.append([new_row0, new_row1])
    
swap_avg_by_hour
    

[[10.8, '19'],
 [7.99, '23'],
 [16.8, '16'],
 [6.75, '22'],
 [21.52, '20'],
 [9.02, '06'],
 [7.17, '04'],
 [7.85, '07'],
 [7.8, '03'],
 [11.38, '01'],
 [14.74, '13'],
 [9.41, '12'],
 [13.44, '10'],
 [8.13, '00'],
 [23.81, '02'],
 [38.59, '15'],
 [11.05, '11'],
 [13.23, '14'],
 [10.25, '08'],
 [13.2, '18'],
 [5.58, '09'],
 [10.09, '05'],
 [11.46, '17'],
 [16.01, '21']]

In [23]:
sorted_swap = sorted(swap_avg_by_hour, reverse = True)
sorted_swap

[[38.59, '15'],
 [23.81, '02'],
 [21.52, '20'],
 [16.8, '16'],
 [16.01, '21'],
 [14.74, '13'],
 [13.44, '10'],
 [13.23, '14'],
 [13.2, '18'],
 [11.46, '17'],
 [11.38, '01'],
 [11.05, '11'],
 [10.8, '19'],
 [10.25, '08'],
 [10.09, '05'],
 [9.41, '12'],
 [9.02, '06'],
 [8.13, '00'],
 [7.99, '23'],
 [7.85, '07'],
 [7.8, '03'],
 [7.17, '04'],
 [6.75, '22'],
 [5.58, '09']]

In [27]:
# Sort the values and print the the 5 hours with the highest average comments.

print("Top 5 Hours for 'Ask HN' Comments")

for hour in sorted_swap[:5]:
    print ("{}:00   :   {} average comments per post".format(hour[1], hour[0]) )

Top 5 Hours for 'Ask HN' Comments
15:00   :   38.59 average comments per post
02:00   :   23.81 average comments per post
20:00   :   21.52 average comments per post
16:00   :   16.8 average comments per post
21:00   :   16.01 average comments per post


The hour that receives the most comments  on average per post is 15:00, averaging 38.59 comments per post. There's more than 50% increase in the number of comments between the hours with the highest and second highest average number of comments.

According to the data set [documentation](https://www.kaggle.com/hacker-news/hacker-news-posts/home), the timezone used is Eastern Time in the US. So, we could also write 15:00 as 3:00 pm EST.


## Conclusion

We analysed ask posts and show posts to determine which type of post and time received the most comments on average. Based on our analysis, to maximise the amount of comments a post receives, we'd recommend the post be categorised as ask post and created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est).

It should be noted that the data set we analyzed excluded posts without any comments. Given that, it's more accurate to say that of the posts that received comments. Majority of ask posts were created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est) and received the most comments on average.